In [1]:
# =============================================================================
# CELL 1: Setup and Directory Structure
# =============================================================================

import sys
from pathlib import Path

# Setup paths
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Configuration helper
def create_config(overrides=None):
    from config import PipelineConfig
    config = PipelineConfig(overrides)
    config.project_root = project_root
    config.documents_root = project_root / "documents"
    config.literature_folder = config.documents_root / "literature"
    config.your_work_folder = config.documents_root / "your_work"
    config.biblio_folder = config.documents_root / "biblio"
    config.current_drafts_folder = config.documents_root / "current_drafts"
    config.cache_file = project_root / "physics_knowledge_base.pkl"
    config._create_directories()
    return config

# Create configuration and directories
config = create_config()

print("📁 YOUR PHYSICS PIPELINE DIRECTORY STRUCTURE")
print("=" * 60)
print(f"Project Root: {project_root}")
print()

# Show where to put files
folders = {
    "biblio": "Put your Zotero .bib files here",
    "literature": "Downloaded papers go here (auto-created)",
    "your_work": "Your own published papers",
    "current_drafts": "Your current drafts"
}

for folder_name, description in folders.items():
    folder_path = config.documents_root / folder_name
    exists = "✅" if folder_path.exists() else "❌"
    file_count = len(list(folder_path.iterdir())) if folder_path.exists() else 0
    
    print(f"{exists} {folder_name}/ - {description}")
    print(f"   📍 Location: {folder_path}")
    print(f"   📊 Current files: {file_count}")
    print()

print("🎯 TO GET STARTED:")
print("1. Export .bib files from Zotero → Save to documents/biblio/")
print("2. Copy your papers → Save to documents/your_work/")
print("3. Run the commands below!")

📁 YOUR PHYSICS PIPELINE DIRECTORY STRUCTURE
Project Root: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis

✅ biblio/ - Put your Zotero .bib files here
   📍 Location: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/biblio
   📊 Current files: 2

✅ literature/ - Downloaded papers go here (auto-created)
   📍 Location: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature
   📊 Current files: 274

✅ your_work/ - Your own published papers
   📍 Location: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/your_work
   📊 Current files: 10

✅ current_drafts/ - Your current drafts
   📍 Location: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/current_drafts
   📊 Current files: 2

🎯 TO GET STARTED:
1. Export .bib files from Zotero → Save to documents/biblio/
2. Copy your papers → Save to documents/your_work/
3. Run

In [2]:
# =============================================================================
# CELL 2: Check for Existing .bib Files
# =============================================================================

# Check what .bib files you already have
biblio_folder = config.biblio_folder
bib_files = list(biblio_folder.glob("*.bib"))

print("📚 CHECKING FOR EXISTING .BIB FILES")
print("=" * 40)

if bib_files:
    print(f"✅ Found {len(bib_files)} .bib file(s):")
    for bib_file in bib_files:
        file_size = bib_file.stat().st_size / 1024  # KB
        print(f"   📄 {bib_file.name} ({file_size:.1f} KB)")
    
    # Quick preview of first .bib file
    first_bib = bib_files[0]
    try:
        content = first_bib.read_text()
        lines = content.split('\n')[:10]
        print(f"\n📖 Preview of {first_bib.name}:")
        for line in lines:
            if line.strip():
                print(f"   {line[:60]}...")
                break
    except Exception as e:
        print(f"   Could not preview file: {e}")
        
    print(f"\n🚀 Ready to download literature!")
    
else:
    print("📭 No .bib files found")
    print("\n💡 TO ADD .BIB FILES:")
    print("1. Open Zotero")
    print("2. Select your collection")  
    print("3. Right-click → Export Collection")
    print("4. Choose 'BibTeX' format")
    print(f"5. Save to: {biblio_folder}")

📚 CHECKING FOR EXISTING .BIB FILES
✅ Found 1 .bib file(s):
   📄 DynamicsConditionalEnsembles_small.bib (117.5 KB)

📖 Preview of DynamicsConditionalEnsembles_small.bib:
   @article{AbhinavDeshpande2022TightBoundsConvergence,...

🚀 Ready to download literature!


In [3]:
# =============================================================================
# CELL 3: Download Literature from .bib Files (IF YOU HAVE THEM)
# =============================================================================

# Only run this if you have .bib files
bib_files = list(config.biblio_folder.glob("*.bib"))

if bib_files:
    print("🚀 DOWNLOADING LITERATURE FROM ARXIV")
    print("=" * 45)
    
    from src.downloaders import LiteratureDownloader
    
    # Use the first .bib file (or modify to use a specific one)
    bib_file = bib_files[0]
    print(f"📖 Using: {bib_file.name}")
    
    # Initialize downloader
    downloader = LiteratureDownloader(
        output_directory=config.literature_folder,
        delay_between_downloads=config.download_delay,
        arxiv_config=config.get_arxiv_config()
    )
    
    print("⏳ Starting download... (this may take several minutes)")
    print("💡 The system will:")
    print("   1. Parse your .bib file")
    print("   2. Search for papers on arXiv")
    print("   3. Download PDFs and TEX files")
    print("   4. Generate a detailed report")
    print()
    
    # Download papers
    results = downloader.download_from_bibtex(bib_file)
    
    print(f"\n✅ DOWNLOAD COMPLETE!")
    print(f"   📥 Successfully downloaded: {len(results['successful'])}")
    print(f"   ❌ Not found: {len(results['failed'])}")
    
    if results['successful']:
        print(f"\n📄 Sample downloads:")
        for i, result in enumerate(results['successful'][:3]):
            paper = result.paper_metadata
            print(f"   {i+1}. {paper.title[:50]}...")
            print(f"      arXiv: {result.search_result.arxiv_id}")
            
else:
    print("⏭️ SKIPPING DOWNLOAD - No .bib files found")
    print("Add .bib files to documents/biblio/ first!")

🚀 DOWNLOADING LITERATURE FROM ARXIV
📖 Using: DynamicsConditionalEnsembles_small.bib
2025-06-02 15:01:15 - physics_pipeline.src.downloaders.bibtex_parser - INFO - BibTeX parser initialized
2025-06-02 15:01:15 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - ArXiv searcher initialized
2025-06-02 15:01:15 - physics_pipeline.src.downloaders.literature_downloader - INFO - Literature downloader initialized with output: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature
⏳ Starting download... (this may take several minutes)
💡 The system will:
   1. Parse your .bib file
   2. Search for papers on arXiv
   3. Download PDFs and TEX files
   4. Generate a detailed report

2025-06-02 15:01:15 - physics_pipeline.src.downloaders.literature_downloader - INFO - Starting download process from /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/biblio/DynamicsConditionalEnsembles_small.bib
2025-06-02 15:01:15 - p

2025-06-02 15:01:38 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 1808.05949
2025-06-02 15:01:38 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 7/80: Many-Body Quantum Chaos and Space-time Translation...
2025-06-02 15:01:38 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Searching for paper: Many-Body Quantum Chaos and Space-time Translation...
2025-06-02 15:01:40 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 2109.04475
2025-06-02 15:01:40 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/2109.04475.pdf
2025-06-02 15:01:41 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/2109.04475.tex
2025-06-02 15:01:41 - physics_pipel

📡 Making Google API request with query: Random Quantum Circuits Are Approximate 2-Designs...
📊 Google returned 5 results
2025-06-02 15:02:08 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 0802.1919
2025-06-02 15:02:09 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/0802.1919.pdf
2025-06-02 15:02:09 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/0802.1919.tex
2025-06-02 15:02:09 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX downloaded and extracted: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/0802.1919.tex
2025-06-02 15:02:10 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 0802.1919
2025-06-02 15:

2025-06-02 15:02:46 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/1411.1768.pdf
2025-06-02 15:02:46 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/1411.1768.tex
2025-06-02 15:02:46 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX downloaded and extracted: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/1411.1768.tex
2025-06-02 15:02:47 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 1411.1768
2025-06-02 15:02:47 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 23/80: Numerical Evaluation of Convex-Roof Entanglement M...
2025-06-02 15:02:47 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - 

2025-06-02 15:03:16 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/2207.14280.pdf
2025-06-02 15:03:18 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/2207.14280.tex
2025-06-02 15:03:18 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX downloaded and extracted: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/2207.14280.tex
2025-06-02 15:03:19 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 2207.14280
2025-06-02 15:03:19 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 30/80: Classicality, the Ensemble Interpretation, and Dec...
2025-06-02 15:03:19 - physics_pipeline.src.downloaders.arxiv_searcher - INF

...

2025-06-02 15:05:19 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 2207.00025
2025-06-02 15:05:19 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/2207.00025.pdf
2025-06-02 15:05:19 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/2207.00025.tex
2025-06-02 15:05:19 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX downloaded and extracted: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/2207.00025.tex
2025-06-02 15:05:20 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 2207.00025
2025-06-02 15:05:20 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 55/80: Emergent Quantum Sta

2025-06-02 15:05:54 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 1106.4292
2025-06-02 15:05:54 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/1106.4292.pdf
2025-06-02 15:05:55 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/1106.4292.tex
2025-06-02 15:05:55 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX downloaded and extracted: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/1106.4292.tex
2025-06-02 15:05:56 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 1106.4292
2025-06-02 15:05:56 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 62/80: Steering Is an Essential 

2025-06-02 15:06:26 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 69/80: Measuring Tr$$ on Single Copies of $$ Using Random...
2025-06-02 15:06:26 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Searching for paper: Measuring Tr$$ on Single Copies of $$ Using Random...

🔍 GOOGLE FALLBACK ACTIVATED for: Measuring Tr$$ on Single Copies of $$ Using Random...
2025-06-02 15:06:29 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - 🔍 Google Custom Search API activated for: Measuring Tr$$ on Single Copies of $$ Using Random...
📡 Making Google API request with query: Measuring Tr$$ on Single Copies of $$ Using Random...
📊 Google returned 5 results
2025-06-02 15:06:30 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 1112.1027
2025-06-02 15:06:30 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literat


🔍 GOOGLE FALLBACK ACTIVATED for: Measurement-Induced Phase Transitions in ( d + 1 )...
2025-06-02 15:06:54 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - 🔍 Google Custom Search API activated for: Measurement-Induced Phase Transitions in ( d + 1 )...
📡 Making Google API request with query: Measurement-Induced Phase Transitions in ( d + 1 )...
📊 Google returned 5 results
2025-06-02 15:06:55 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 2411.06648
2025-06-02 15:06:58 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/2411.06648.pdf
2025-06-02 15:06:58 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis/documents/literature/2411.06648.tex
2025-06-02 15:06:58 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX download


📚 LITERATURE DOWNLOAD SUMMARY

📊 OVERALL STATISTICS:
   Total papers processed: 80
   Successfully downloaded: 56
   Failed downloads: 24
   Success rate: 70.0%
   Total time: 373.8 seconds
   Average time per paper: 4.7 seconds

📄 FILE DOWNLOADS:
   PDF files: 56
   TEX files: 48

🔍 SEARCH METHODS:
   title_high_confidence: 44 papers
   title_with_abstract_validation: 2 papers
   google_search: 10 papers

✅ SUCCESSFUL DOWNLOADS (showing first 5):
   1. Tight Bounds on the Convergence of Noisy Random Circuits to ...
      arXiv: 2112.00716 (confidence: 1.000)
   2. Critical Properties of the Measurement-Induced Transition in...
      arXiv: 1911.00008 (confidence: 1.000)
   3. Exponential Shortcut to Measurement-Induced Entanglement Pha...
      arXiv: 2302.14044 (confidence: 1.000)
   4. Measurement-Induced Topological Entanglement Transitions in ...
      arXiv: 2004.07243 (confidence: 1.000)
   5. Unitary-Projective Entanglement Dynamics...
      arXiv: 1808.05949 (confidence: 1.00

In [ ]:
# =============================================================================
# CELL 4: Build Knowledge Base (WITH PYTORCH FIX)
# =============================================================================

# PYTORCH COMPATIBILITY FIX - Add this first!
import torch

if not hasattr(torch, 'get_default_device'):
    def get_default_device():
        """Fallback implementation for older PyTorch versions."""
        if torch.cuda.is_available():
            return torch.device('cuda')
        else:
            return torch.device('cpu')
    
    torch.get_default_device = get_default_device
    print("✅ Applied PyTorch compatibility fix")

# Now the imports should work
print("\n🏗️ BUILDING KNOWLEDGE BASE")
print("=" * 35)

from src.core import KnowledgeBase

# Initialize knowledge base
kb = KnowledgeBase(
    embedding_model=config.embedding_model,
    chunk_size=config.chunk_size,
    chunk_overlap=config.chunk_overlap
)

print("⏳ Building knowledge base from all documents...")
print("💡 This processes:")
print("   📚 Downloaded literature")
print("   📝 Your own papers") 
print("   ✏️  Current drafts")
print("   🧠 Creates semantic embeddings")
print()

# Build knowledge base
stats = kb.build_from_directories(
    literature_folder=config.literature_folder,
    your_work_folder=config.your_work_folder,
    current_drafts_folder=config.current_drafts_folder
)

print("✅ KNOWLEDGE BASE BUILT!")
print(f"   📚 Total documents: {stats.get('total_documents', 0)}")
print(f"   ✅ Successfully processed: {stats.get('successful_documents', 0)}")
print(f"   🧩 Total text chunks: {stats.get('total_chunks', 0)}")
print(f"   📊 Total words: {stats.get('total_words', 0):,}")

# Show source breakdown
source_breakdown = stats.get('source_breakdown', {})
if source_breakdown:
    print(f"\n📁 Source breakdown:")
    for source_type, info in source_breakdown.items():
        count = info.get('successful', 0) if isinstance(info, dict) else info
        print(f"   {source_type}: {count} documents")

# Save knowledge base
kb.save_to_file(config.cache_file)
print(f"\n💾 Knowledge base saved to: {config.cache_file}")

# Store for next cell
globals()['knowledge_base'] = kb

In [ ]:
# =============================================================================
# CELL 5: Test Literature-Aware Chat (IF YOU HAVE API KEY)
# =============================================================================

print("\n🤖 TESTING LITERATURE-AWARE CHAT")
print("=" * 40)

try:
    config.validate_api_keys()
    
    from src.chat import LiteratureAssistant
    
    # Create literature assistant
    assistant = LiteratureAssistant(
        knowledge_base=kb,
        anthropic_api_key=config.anthropic_api_key,
        chat_config=config.get_chat_config()
    )
    
    print("✅ Literature assistant created!")
    print(f"📊 Knowledge base: {kb.get_statistics().get('total_documents', 0)} documents")
    
    # Simple chat function
    def chat(question, show_sources=True):
        """Simple chat with the literature assistant."""
        print(f"\n🧑‍🔬 Question: {question}")
        print("🤖 Assistant: ", end="", flush=True)
        
        try:
            response = assistant.ask(question)
            print(response.content)
            
            if show_sources and response.sources_used:
                print(f"\n📚 Sources: {', '.join(response.sources_used)}")
            
        except Exception as e:
            print(f"Error: {e}")
        
        print("\n" + "-" * 50)
    
    print("\n💬 READY FOR CHAT!")
    print("💡 Try these commands:")
    print("   chat('What papers do I have in my knowledge base?')")
    print("   chat('What are the main research topics?')")
    print("   chat('Explain quantum entanglement')")
    
    # Store chat function for easy use
    globals()['chat'] = chat
    globals()['assistant'] = assistant
    
except ValueError as e:
    print("⚠️  Cannot test chat - API key not set")
    print("💡 Set your ANTHROPIC_API_KEY environment variable to test chat")

# =============================================================================
# CELL 6: Summary and Next Steps
# =============================================================================

print("\n🎉 PHYSICS PIPELINE SETUP COMPLETE!")
print("=" * 45)

# Final statistics
bib_count = len(list(config.biblio_folder.glob("*.bib")))
lit_count = len(list(config.literature_folder.iterdir())) if config.literature_folder.exists() else 0
work_count = len(list(config.your_work_folder.iterdir())) if config.your_work_folder.exists() else 0

print(f"📊 FINAL STATUS:")
print(f"   📚 .bib files: {bib_count}")
print(f"   📄 Downloaded papers: {lit_count}")
print(f"   📝 Your papers: {work_count}")
print(f"   🧠 Knowledge base: {'✅ Built' if 'knowledge_base' in globals() else '❌ Not built'}")
print(f"   🤖 Chat ready: {'✅ Yes' if 'chat' in globals() else '❌ Need API key'}")

print(f"\n🚀 WHAT YOU CAN DO NOW:")
if 'chat' in globals():
    print("   • chat('your question here') - Ask the AI assistant")
    print("   • assistant.synthesize_literature('topic') - Get research synthesis")
    print("   • assistant.help_with_writing('task') - Get writing help")
else:
    print("   • Add .bib files and run cells above")
    print("   • Set ANTHROPIC_API_KEY for chat functionality")

print(f"\n📁 KEY LOCATIONS:")
print(f"   📚 Add .bib files: {config.biblio_folder}")
print(f"   📝 Add your papers: {config.your_work_folder}")
print(f"   📄 Downloaded papers: {config.literature_folder}")
print(f"   💾 Knowledge base cache: {config.cache_file}")

print("\n🎯 Your physics literature synthesis pipeline is ready! 🔬📚")

In [ ]:
# Test 1: Overview of your collection
chat("What papers do I have in my knowledge base?")

In [ ]:
chat("Do you know what measurement-induced entanglement phase transitions are?")